In [1]:
import  pandas as pd
from openpyxl import load_workbook

# Define file path
df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Innova Sales Report\KE24.xlsx")
file_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Innova Sales Report\Mapping.xlsx"

# Load Excel file
xls = pd.ExcelFile(file_path)

# Read "Sales Office" and "Innova Group" sheets
df_region = pd.read_excel(xls, sheet_name="Sales Office", header=0)
df_group = pd.read_excel(xls, sheet_name="Innova Group", header=0)

# Read Customer masterdata
df_customer=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Customer Master Data _Etex.XLSX")
df_customer=df_customer[['Customer','Sales Office']].drop_duplicates(subset='Customer')
df_customer['Customer']=df_customer['Customer'].astype('string')
df['Customer']=df['Customer'].astype('string')

df=df.merge(df_customer, on='Customer', how='left')

df=df.merge(df_region, on='Sales Office', how='left')

df_group['Product']=df_group['Product'].astype('string')
df['Product']=df['Product'].astype('string')
df=df.merge(df_group, on='Product', how='left')



In [3]:
# Load the workbook
final_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\19.Business Systems\Innova Sales Report\Innova Sales.xlsx"
wb = load_workbook(final_path)
ws = wb["Sheet1"]

# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)
df = df.astype(object).where(pd.notna(df), None)


# Write updated DataFrame back to "Raw Data" (including headers)
for c_idx, column in enumerate(df.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(final_path)

